In [ ]:
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt

# Tutorial 3 - Renormalization and scale invariance

## Exercise 1 - The Koch curve

The idea of renormalization is closely related to the concept of scale invariance. In this first exercise we will explore the concept of scale invariance and how it can be applied to a simple system, the ["Koch curve"](https://en.wikipedia.org/wiki/Koch_snowflake), which is a fractal curve that exhibits self-similarity at different scales. 

The following python snippet shows how to generate the Koch curve using a recursive algorithm. The Koch curve is defined by starting with a straight line segment and recursively replacing each segment with a specific pattern that introduces new points, creating a fractal structure. In this way, the curve looks the same at any scale, i.e. it is invariant under dilatation.

Similary, the renormalization group tells us how physical parameters, like coupling constants, change when we change the energy resolution. At the so-called fixed point, physical parameters do not change anymore, i.e. they are invariant under dilatation, just like the Koch curve.

In [ ]:
# Define the generating functions for the Koch curve
def rotation(angle: float, x: npt.ArrayLike, r: float) -> npt.ArrayLike:
    """Rotate points by an angle."""
    rot = (
        np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
        / r
        @ x.T
    )
    return rot.T


def koch_curve(x: npt.ArrayLike) -> npt.ArrayLike:
    """Add all points in this iteration."""
    r = 3
    # scale by 3
    X1 = x / r
    # rotate by 60°
    X2 = rotation(np.pi / 3, x, r) + [1 / r, 0]
    # rotate by - 60°
    X3 = rotation(-np.pi / 3, x, r) + np.array(
        [np.cos(np.pi / 3), np.sin(np.pi / 3) / r]
    )
    # scale by 3
    X4 = x / r + [2 / r, 0]
    return [X1, X2, X3, X4]


# let's see it in action!
L = 1.0
start_points = np.array([[0, 0], [L, 0]])
iterations = 4
fig, axes = plt.subplots(
    1 + iterations // 3,
    3,
    figsize=(15, 5 * (1 + iterations // 3)),
    clear=True,
    layout="tight",
)
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
# iterate the prescription
for i, ax, col in zip(range(0, iterations + 1), axes.flat, colors):
    if i == 0:
        # at iteration 0 we have a simple line
        all_points = [start_points]
    else:
        new_points = []
        # loop in present point
        for old_point in all_points:
            new_points.extend(koch_curve(old_point))
        # update the list of poitns
        all_points = new_points
    for pts in all_points:
        ax.plot(pts[:, 0], pts[:, 1], color=col)
    ax.set_title(f"Iteration {i}")
    ax.set_ylim(0, L)

Now, imposing invariance under infinitesimal dilatations lets us derive its corresponding Renormalization Group Equation (RGE). For this we consider the function $N(\ell)$ that describes the number of line segments at a given segment length $\ell$. 

### Question 1A

Find the number of line segments $N_n$ and the length of each segment $\ell_n$ at the $n$-th iteration of the Koch curve.

### Question 1B

Express $N$ in terms of $\ell$ by eliminating $n$ and show that $N(\ell)$ scales under rescalings by $\lambda$ as

$$ N(\lambda \ell) = \lambda^{-\gamma} N(\ell) $$

where $\gamma$ is the scaling dimension. What value of $\gamma$ do you find? 

### Question 1C

Show that for infinitesimal transformations $\lambda \ell$ = $\ell + \delta\ell$ the scaling relation can be written as a differential equation:

$$ \ell \frac{d N}{d \ell} = -\gamma N $$

where $\gamma$ is the scaling dimension you found in the previous exercise. 


Similarly in QCD once the renormalized fields are defined in terms of the bare quantities and imposing this 
procedure holds at any arbitrary scale, it is possible to obtain a RGE. In the following tutorial, we will analyze how this particular differential equation can be applied to describe the dependency 
on the energy scale of the strong coupling $a_s$.

## Exercise 2 - The strong coupling running

Choosing the common normalization $a_s = \alpha_s/(4\pi)$ we can write down the the RGE of the QCD strong coupling by:

$$ \mu^2 \frac{d a_s}{d \mu^2} = \beta(a_s) = - \sum_{j=0} \beta_j a_s^{j+2} $$

where $\beta(a_s)$ is calculated perturbatively using Feynman diagrams.

Let's take a closer look to that equation:
- it is a first order ordinary differential equation; so, to solve it we need a boundary value $a_s(\mu_0^2)$ given at a reference scale $\mu_0$
- the dimensions check out: $a_s$ is a dimensionless quantity, so are all $\beta_j$ and the differential operators are dimensionless as well

Note that our definition of the coefficients $\beta_j$ pulls out a minus sign (other people use other conventions), such that $\beta_0 > 0$ in our convention. The strong coupling has a negative derivative (lhs), meaning that it it becomes smaller at larger scales - people refer to this as [asymptotic freedom](https://en.wikipedia.org/wiki/Asymptotic_freedom) and this is the reason why we do QCD experiments at high energy colliders: QCD becomes perturbative in the high energy region.

On the other hand, the beta function is negative, so there is no zero derivative, and hence there is no fixed point in QCD at any finite scale (but of course at $\mu^2 = \infty$).

### Exercise 2A

Solve the RGE at leading order, $j=0$, and show that the solution is given by

$$ a_s(\mu^2) = \frac{a_s(\mu_0^2)}{1 + \beta_0 a_s(\mu_0^2) \log(\mu^2/\mu_0^2)} $$

and expand the solution up to quadratic order in $a_s(\mu_0^2)$. What do you notice when $\mu$ and $\mu_0$ are far apart?



The point where the denominator becomes 0 is called the [Landau pole](https://en.wikipedia.org/wiki/Landau_pole) - since the coupling would become infinite this is surely not a region where QCD can be treated as a perturbative QFT.

### Exercise 2B

The [eko](https://github.com/NNPDF/eko) library was developed to solve the RGE of the strong coupling - let's use it to plot the strong coupling!

In [ ]:
from eko.couplings import CouplingEvolutionMethod, Couplings, CouplingsInfo
from eko.quantities.heavy_quarks import QuarkMassScheme
from eko.beta import beta_qcd

# setup the boundary conditions
ref = CouplingsInfo(
    alphas=0.1180,  # reference scale for QCD coupling
    alphaem=0.007496,  # reference scale for QED coupling
    scale=91.00,  # reference scale
    num_flavs_ref=5,  # number of active flavors at the reference scale
    max_num_flavs=6,  # max flavors allowed
)
order = (1, 0)  # Set perturbative order (QCD, QED)
# create a strong coupling object
sc = Couplings(
    couplings=ref,
    order=order,
    method=CouplingEvolutionMethod.EXACT,  # Set solution method EXACT or EXPANDED
    masses=np.array([1.51, 4.92, 172.5]) ** 2,  # Set heavy quarks masses
    hqm_scheme=QuarkMassScheme.POLE,  # Use POLE masses or MSBAR ?
    thresholds_ratios=np.array([1.0, 1.0, 1.0])
    ** 2,  # Set heavy quarks threshold scales
)


# compute some values of a_s
Q2_grid = np.geomspace(1, 1e4, 100)
as_vals_nf_5 = [sc.a_s(Q2, 5) for Q2 in Q2_grid]


# plot the running strong coupling
plt.close()
plt.plot(Q2_grid, as_vals_nf_5)


plt.title("LO strong coupling")
plt.xscale("log")
plt.ylabel(r"$a_s(\mu^2)$")
plt.xlabel(r"$\mu^2$ [GeV²]")
plt.show()

Repeat this for 4 active flavors and plot the strong coupling again, together with the previous one at `num_flavs_ref=5`. Comment on what you see and argue which one is the correct one, if any.

### Flavour thresholds and matching conditions

The problem is that the beta function only holds for a given number of light/active flavors $n_f$ since the beta coefficients depend on $n_f$.
However, in practice the assumptions on how many flavors can be considered light is a dynamic choice - we want to change that according to the scale.
The scale at which we change from one flavor regime to another we refer to as matching scale $\mu_h$ - and a natural choice for the matching scale are the heavy quark masses. In our example above this means that above $m_b$ we solve the beta function with $n_f=5$ (green), but below with with $n_f=4$ (blue).
Note, that our default solution (red) first follows the green curve ($n_f=5$) and then the blue curve ($n_f=4$) and in the opposite region the two do not coincide. Moreover for $\mu=m_b$ all three curves agree.

In [ ]:
# Let's have a closer look to the region around the bottom mass
mb = 4.92
Q2_grid = np.geomspace(mb**2 * (1.0 - 0.2), mb**2 * (1.0 + 0.2), 100)

# Let's plot again the strong coupling running
plt.close()
as_lo_qcd = [sc.a_s(Q2) for Q2 in Q2_grid]
plt.plot(Q2_grid, as_lo_qcd, "r", label="default")
# We add two more line, on which we comment in a second
plt.plot(Q2_grid, [sc.a_s(Q2, 4) for Q2 in Q2_grid], "b--", label="$n_f=4$")
plt.plot(Q2_grid, [sc.a_s(Q2, 5) for Q2 in Q2_grid], "g--", label="$n_f=5$")
plt.title("LO strong coupling")
plt.xscale("log")
plt.ylabel(r"$a_s(\mu^2)$")
plt.xlabel(r"$\mu^2$ [GeV²]")
plt.legend()
plt.show()



The exact procedure on how to join two different flavor regimes is refered to as **matching condition**. At LO (as shown here), you only need to change the
coefficients of the beta function, but at higher perturbative orders you may need to do more (i.e. finite corrections have to be applied). Putting the quark thresholds to coincide with the quark masses is a choice and we can in practice choose any value.

### Higher order corrections

Now we start to consider higher order corrections, i.e. more terms in the beta function, i.e. $\beta_1,\beta_2,\ldots$

In [ ]:
# Let's compare the exact and the expanded solution
sc_lo = Couplings(
    couplings=ref,
    order=(1, 0),  # Set perturbative order (QCD, QED)
    method=CouplingEvolutionMethod.EXACT,  # Set solution method EXACT or EXPANDED
    masses=np.array([1.51, 4.92, 172.5]) ** 2,  # Set heavy quarks masses
    hqm_scheme=QuarkMassScheme.POLE,  # Use POLE masses or MSBAR ?
    thresholds_ratios=np.array([1.0, 1.0, np.inf])
    ** 2,  # Set heavy quarks threshold scales
)

sc_nlo = Couplings(
    couplings=ref,
    order=(2, 0),  # Set perturbative order (QCD, QED)
    method=CouplingEvolutionMethod.EXACT,  # Set solution method EXACT or EXPANDED
    masses=np.array([1.51, 4.92, 172.5]) ** 2,  # Set heavy quarks masses
    hqm_scheme=QuarkMassScheme.POLE,  # Use POLE masses or MSBAR ?
    thresholds_ratios=np.array([1.0, 1.0, np.inf])
    ** 2,  # Set heavy quarks threshold scales
)

# we collect first the data ...
Q2_grid = np.linspace(20**2, 170**2, 100)

lo_qcd = np.array([sc_lo.a_s(Q2) for Q2 in Q2_grid])
nlo_qcd = np.array([sc_nlo.a_s(Q2) for Q2 in Q2_grid])

# and then plot
plt.close()
fig = plt.figure()
fig.suptitle("strong coupling")
ax0 = fig.add_subplot(3, 1, (1, 2))
ax0.plot(Q2_grid, lo_qcd, label="LO QCD")
ax0.plot(Q2_grid, nlo_qcd, label="NLO QCD")


ax0.legend()
ax0.set_ylabel(r"$a_s(\mu^2)$")
ax1 = fig.add_subplot(3, 1, 3, sharex=ax0)
ax1.plot(Q2_grid, nlo_qcd / lo_qcd - 1)
ax1.set_ylabel(r"rel. difference")
ax1.set_xlabel(r"$\mu^2$ [GeV²]")
ax1.set_xscale("log")
fig;

## Exercise 2C
* Can you plot now the running of $a_{em}$ and $a_s$ with $QED \otimes QCD$ ?
* What is the main difference with respect to the $QCD$ one ?

Fill in the gaps:

In [ ]:
# setup the boundary conditions
ref = CouplingsInfo(
    alphas=0.1180,  # reference scale for QCD coupling
    alphaem=0.007496,  # reference scale for QED coupling
    scale=91.00,  # reference scale
    num_flavs_ref=5,  # number of active flavors at the reference scale
    max_num_flavs=6,  # max flavors allowed
    em_running=True,
)
# create a strong coupling object with the exact solution method of the evolution
sc = Couplings(
)


# Create definitions for a_em and a_s from the strong coupling object

def a_em(Q2: float) -> float:
    """Return :math:`a_{em} = \alpha_{em} / (4 \pi)`."""
    
def a_s(Q2: float) -> float:
    """Return :math:`a_{s} = \alpha_{s} / (4 \pi)`."""


# compute some values of a_s and a_em 
Q2_grid = np.geomspace(1, 1e4, 100)
aem_vals = 
as_vals = 


# plot the running EM coupling (with QCD corrections)


# plot the running strong coupling (with QED corrections)
